In [ ]:
# Dependencies
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report
import joblib

In [ ]:
# Read the CSV
df = pd.read_csv("../cleaned_data/cleaned_house_crime_school.csv")
df

# Select features (columns)

In [ ]:
# Set features to be used as X values.
X = df.drop(["Price", "Suburb", "Date", "Address", "Type", "Postcode", "CouncilArea", "Lattitude", "Longtitude", "Regionname", "Distance", "CrimeRate"], axis = "columns")
y = df["Price"]
print(X.shape, y.shape)

In [ ]:
# GET housing stats
total_houses = len(df)
max_value = df["Price"].describe()["max"]
min_value = df["Price"].describe()["min"]
print(f"Total houses: {total_houses}")
print(f"Highest price: {max_value}")
print(f"Lowest price: {min_value}")

# Split the data into test and train data using `train_test_split` with test size of 33%

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.33)
X_train

In [ ]:
# Scale the data
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
classifier = LogisticRegression(solver = "newton-cg", multi_class = "auto")
classifier.fit(X_train, y_train)

training_score = classifier.score(X_train, y_train)
base_accuracy = classifier.score(X_test, y_test)

print(f"LogisticRegression training Data Score: {training_score}")
print(f"LogisticRegression testing Data Score: {base_accuracy}")

# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [ ]:
# Create the GridSearchCV model
param_grid = {"C": [0.1, 1, 10],
              "max_iter": [1000, 5000, 10000]}
grid = GridSearchCV(classifier, param_grid, error_score = "raise", verbose = 3)

In [ ]:
# Train the model with GridSearch
grid.fit(X_train, y_train)

In [ ]:
print(f"Best grid params: {grid.best_params_}")
print(f"Best grid score: {grid.best_score_}")

# Train Tuned Model

In [ ]:
# Tuned parameters
C = grid.best_params_["C"]
max_iter = grid.best_params_["max_iter"]

# Tuned model
tuned_model = LogisticRegression(solver = "newton-cg",
                                 multi_class = "auto",
                                 C = C,
                                 max_iter = max_iter)
tuned_model.fit(X_train_scaled, y_train)

tuned_model_score = tuned_model.score(X_train_scaled, y_train)
tuned_accuracy = tuned_model.score(X_test_scaled, y_test)

print(f"Training Data Score: {tuned_model_score}")
print(f"Testing Data Score: {tuned_accuracy}")

In [ ]:
# Make predictions with the hypertuned model
predictions = tuned_model.predict(X_test)
classifications = y_test.unique().tolist()

prediction_actual = {"Actual": y_test,
                     "Prediction": predictions}

prediction_df = pd.DataFrame(prediction_actual)
prediction_df = prediction_df.set_index("Actual").reset_index()
prediction_df

In [ ]:
evaluations = {"": ["Base Model", "Tuned Model"],
               "Accuracy": [f"%s" % round(base_accuracy, 3), f"%s" % round(tuned_accuracy, 3)]}

evaluations_df = pd.DataFrame(evaluations)
evaluations_df = evaluations_df.set_index("")

evaluations_df.to_csv("../evaluations/random_forest_eval.csv")
evaluations_df

# Save the model

In [ ]:
filename = "../models/random_forest.sav"
joblib.dump(tuned_model, filename)